<a href="https://colab.research.google.com/github/CharlotteY2003/MIT-Futuremakers/blob/main/News_Headline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import *
from sklearn.model_selection import train_test_split, StratifiedKFold
import itertools
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.optimizers import RMSprop

In [2]:
#for google colab python is not run into computer so can't access through computer - need to upload files to colab

def parse_data(file):
  for l in open(file, 'r'):
    yield json.loads(l)

url = 'Sarcasm_Headlines_Dataset.json'

dataset = list(parse_data(url))

In [3]:
dataset = pd.DataFrame(data=dataset)

In [4]:
sar_det = dataset[dataset.is_sarcastic==1]
sar_det.reset_index(drop=True, inplace=True)
acc_det = dataset[dataset.is_sarcastic == 0]
acc_det.reset_index(drop=True, inplace=True)

In [5]:
def tokenization(data):
  arr = []
  for rows in range(0, data.shape[0]):
    head_txt = data.headline[rows]
    head_txt = head_txt.split(" ")
    arr.append(head_txt)
  
  data_list = list(itertools.chain(*arr))
  return data_list, arr

In [6]:
sar_list, sar_news = tokenization(sar_det)
acc_list, acc_news = tokenization(acc_det)

In [7]:
#Stopwords
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
sar_list_restp = [word for word in sar_list if word.lower() not in stopwords]
acc_list_restp = [word for word in acc_list if word.lower() not in stopwords]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
print("Length of original sarcasm list: {0} words \n"
      "Length of new sarcasm list: {1} words\n".format(len(sar_list), len(sar_list_restp)))
print('=='*46)
print("Length of original accuracy list: {0} words \n"
      "Length of new accuracy list: {1} words\n".format(len(acc_list), len(acc_list_restp)))

Length of original sarcasm list: 115898 words 
Length of new sarcasm list: 87458 words

Length of original accuracy list: 147128 words 
Length of new accuracy list: 103525 words



In [9]:
#Stemming - not great because it disregards semantic meaning of word
stemmer = nltk.stem.SnowballStemmer('english', ignore_stopwords = True)

In [10]:
#Lemminization
lemm = WordNetLemmatizer()

In [13]:
def lemminizor(news, restp):
  arr = []
  for batch in news:
    restp = [word for word in batch if word.lower() not in stopwords]
    lemm = WordNetLemmatizer()
    list_lemm = [lemm.lemmatize(word) for word in restp]
    arr.append(list_lemm)

  return arr

In [ ]:
nltk.download('wordnet')
sar_wost_lemm = lemminizor(sar_news, sar_list_restp)
acc_wost_lemm = lemminizor(acc_news, acc_list_restp)

#Actual Start of NLP NN

In [18]:
y = dataset.is_sarcastic
X = dataset.headline
le = LabelEncoder()
y = le.fit_transform(y)
y = y.reshape(-1,1)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .33, random_state = 42)

In [ ]:
#max_words = 1000
#max_len = 150
#tok = Tokenizer(num_words = max_words)
#tok.fit_on_texts(X_train)
#sequences = tok.texts_to_sequences(X_train)
#sequences_matrix = sequence.pad_sequences(sequences, maxlen = max_len)

In [ ]:
#model = Sequential()
#model.add(Dense(64, activation = 'relu', input_shape = (max_len,)))
#model.add(Embedding(max_words, 50, input_length=max_len))
#model.add(Dense(64, activation = 'relu'))
#model.add(Dropout(.2))
#model.add(Dense(1, activation = 'sigmoid'))
#model.summary()

In [ ]:
seed = 7
np.random.seed(seed)

kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)
for train, test in kfold.split(X,y):
  max_words = 1000
  max_len = 150
  
  model = Sequential()
  model.add(Dense(64, activation = 'relu', input_shape = (max_len,)))
  model.add(Embedding(max_words, 50, input_length=max_len))
  model.add(Dense(64, activation = 'relu'))
  model.add(Dropout(.2))
  model.add(Dense(1, activation = 'sigmoid'))

  
  tok = Tokenizer(num_words = max_words)
  tok.fit_on_texts(X[train])
  sequences = tok.texts_to_sequences(X[train])
  sequences_matrix = sequence.pad_sequences(sequences, maxlen = max_len)

  model.compile(loss = 'binary_crossentropy', optimizer='RMSprop', metrics = ['accuracy'])
  
  model.fit(sequences_matrix, y[train], epochs = 5, batch_size = 100, verbose = 1)

  test_sequences = tok.texts_to_sequences(X[test])
  test_sequences_matrix = sequence.pad_sequences(test_sequences, maxlen = max_len)
  y_pred = model.predict(test_sequences_matrix)
  score = model.evaluate(test_sequences_matrix, y[test])
  print(score)